# KILT-7-new dataset loader
- current dataset loader in FiD does not work well with there are less passages than designated n_context
- need to fix 

# Error Code

- RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 5 but got size 1 for tensor numbe
r 2 in the list.

In [1]:
import torch
import random
import json
import numpy as np
import transformers
from torch.utils.data import DataLoader, SequentialSampler
from pprint import pprint

In [2]:
# class Collator(object):
#     def __init__(self, text_maxlength, tokenizer, answer_maxlength=20):
#         self.tokenizer = tokenizer
#         self.text_maxlength = text_maxlength
#         self.answer_maxlength = answer_maxlength

#     def __call__(self, batch):
#         assert(batch[0]['target'] != None)
#         index = torch.tensor([ex['index'] for ex in batch])
#         target = [ex['target'] for ex in batch]
        
#         target = self.tokenizer.batch_encode_plus(
#             target,
#             max_length=self.answer_maxlength if self.answer_maxlength > 0 else None,
#             pad_to_max_length=True,
#             return_tensors='pt',
#             truncation=True if self.answer_maxlength > 0 else False,
#         )
        
#         target_ids = target["input_ids"]
#         target_mask = target["attention_mask"].bool()
#         target_ids = target_ids.masked_fill(~target_mask, -100)
        
#         def append_question(example):
#             if example['passages'] is None:
#                 return [example['question']]
#             return [example['question'] + " " + t for t in example['passages']]
#         text_passages = [append_question(example) for example in batch]
#         passage_ids, passage_masks = encode_passages(text_passages,
#                                                      self.tokenizer,
#                                                      self.text_maxlength)

#         return (index, target_ids, target_mask, passage_ids, passage_masks)


In [33]:
class Collator(object):
    def __init__(self, text_maxlength, tokenizer, n_context, answer_maxlength=20):
        self.tokenizer = tokenizer
        self.text_maxlength = text_maxlength
        self.answer_maxlength = answer_maxlength
        self.n_context = n_context

    def __call__(self, batch):
        assert(batch[0]['target'] != None)
        index = torch.tensor([ex['index'] for ex in batch])
        target = [ex['target'] for ex in batch]
        
#         print(f'batch : {len(batch)}')
#         print('target')
#         pprint(target)
        target = self.tokenizer.batch_encode_plus(
            target,
            max_length=self.answer_maxlength if self.answer_maxlength > 0 else None,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True if self.answer_maxlength > 0 else False,
        )
        
#         print(f'target')
#         pprint(target)
        target_ids = target["input_ids"]
#         print(f'target_ids : {target_ids}')
#         print(f'target_attention_mask')
#         print(f'{target["attention_mask"]}')
        target_mask = target["attention_mask"].bool()
#         print(f'target_mask : {target_mask}')
        
        target_ids = target_ids.masked_fill(~target_mask, -100)
#         print(f'target_ids : {target_ids}')

        
        
#         print(example)
#         pprint(example)
        def append_question(example):
            if example['passages'] is None:
                return [example['question']]
            return [example['question'] + " " + t for t in example['passages']]
        
        text_passages = []
        
        for example in batch:
#             if len(example) > 5:
#                 print('example')
#                 pprint(example)
            text_passages.append(append_question(example))
#         text_passages = [append_question(example) for example in batch]
        
#         print('text_passages')
#         pprint(text_passages)
        passage_ids, passage_masks = encode_passages(text_passages,
                                                     self.tokenizer,
                                                     self.text_maxlength,
                                                     self.n_context,)
#         print(f'-----index------')
#         pprint(index)
#         print(f'-----target_ids------')
#         print(target_ids.shape)
#         pprint(target_ids)
#         print(f'-----target_mask------')
#         print(target_mask.shape)
#         pprint(target_mask)
#         print(f'-----passage_ids------')
#         print(passage_ids.shape)
#         pprint(passage_ids)
#         print(f'-----passage_masks------')
#         print(passage_masks.shape)
#         pprint(passage_masks)
        return (index, target_ids, target_mask, passage_ids, passage_masks)


In [34]:
def load_data(data_path=None, global_rank=-1, world_size=-1):
    assert data_path
    if data_path.endswith('.jsonl'):
        data = open(data_path, 'r')
    elif data_path.endswith('.json'):
        with open(data_path, 'r') as fin:
            data = json.load(fin)
    examples = []
    for k, example in enumerate(data):
        if global_rank > -1 and not k%world_size==global_rank:
            continue
        if data_path is not None and data_path.endswith('.jsonl'):
            example = json.loads(example)
        if not 'id' in example:
            example['id'] = k
        for c in example['ctxs']:
            if not 'score' in c:
                c['score'] = 1.0 / (k + 1)
        examples.append(example)
    ## egrave: is this needed?
    if data_path is not None and data_path.endswith('.jsonl'):
        data.close()

    return examples

In [35]:
# def encode_passages(batch_text_passages, tokenizer, max_length):
#     passage_ids, passage_masks = [], []
#     for k, text_passages in enumerate(batch_text_passages):
#         p = tokenizer.batch_encode_plus(
#             text_passages,
#             max_length=max_length,
#             pad_to_max_length=True,
#             return_tensors='pt',
#             truncation=True
#         )
#         passage_ids.append(p['input_ids'][None])
#         passage_masks.append(p['attention_mask'][None])

#     passage_ids = torch.cat(passage_ids, dim=0)
#     passage_masks = torch.cat(passage_masks, dim=0)
#     return passage_ids, passage_masks.bool()

In [36]:
def encode_passages(batch_text_passages, tokenizer, max_length, n_context):
    passage_ids, passage_masks = [], []
    for k, text_passages in enumerate(batch_text_passages):
        p = tokenizer.batch_encode_plus(
            text_passages,
            max_length=max_length,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True
        )
        
        cur_ctx = p['input_ids'].shape[0]
        print("p['input_ids'] type")
        print(p['input_ids'].dtype)
        print('--------')
        if cur_ctx < n_context:
            repl_lst = []
            while cur_ctx < n_context:
                cur_ctx += 1
                repl_lst.append(torch.zeros_like(p['input_ids']))
            repli_tensor = torch.cat(repl_lst, dim=0)
            print("repli_tensor type")
            print(repli_tensor.dtype)
            p['input_ids'] = torch.cat([p['input_ids'], repli_tensor], dim = 0)
            
        cur_ctx = p['attention_mask'].shape[0]
        
        print("p['attention_mask'] type")
        print(p['attention_mask'].dtype)
        print('--------')
        if cur_ctx < n_context:
            repl_lst = []
            while cur_ctx < n_context:
                cur_ctx += 1
                repl_lst.append(torch.zeros_like(p['attention_mask']))
            repli_tensor = torch.cat(repl_lst, dim=0)
            print("repli_tensor type")
            print(repli_tensor.dtype)
            print('--------')
            p['attention_mask'] = torch.cat([p['attention_mask'], repli_tensor], dim = 0)
        
#         print(f'cur_ctx < n_context : {cur_ctx < n_context}')
#         print(p['input_ids'].shape)
#         print(p['attention_mask'].shape)
        
        passage_ids.append(p['input_ids'][None])
        passage_masks.append(p['attention_mask'][None])

    passage_ids = torch.cat(passage_ids, dim=0)
    passage_masks = torch.cat(passage_masks, dim=0)
    return passage_ids, passage_masks.bool()

In [37]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,
                 data,
                 n_context=None,
                 question_prefix='question:',
                 title_prefix='title:',
                 passage_prefix='context:'):
        self.data = data
        self.n_context = n_context
        self.question_prefix = question_prefix
        self.title_prefix = title_prefix
        self.passage_prefix = passage_prefix
        self.sort_data()

    def __len__(self):
        return len(self.data)

    def get_target(self, example):
        if 'target' in example:
            target = example['target']
            return target + ' </s>'
        elif 'answers' in example:
            return random.choice(example['answers']) + ' </s>'
        else:
            return None

    def __getitem__(self, index):
        example = self.data[index]
        question = self.question_prefix + " " + example['question']
        target = self.get_target(example)

        if 'ctxs' in example and self.n_context is not None:
            f = self.title_prefix + " {} " + self.passage_prefix + " {}"
            contexts = example['ctxs'][:self.n_context]
            passages = [f.format(c['title'], c['text']) for c in contexts]
            scores = [float(c['score']) for c in contexts]
            scores = torch.tensor(scores)
            # TODO(egrave): do we want to keep this?
            if len(contexts) == 0:
                contexts = [question]
        else:
            passages, scores = None, None


        return {
            'index' : index,
            'question' : question,
            'target' : target,
            'passages' : passages,
            'scores' : scores
        }

    def sort_data(self):
        if self.n_context is None or not 'score' in self.data[0]['ctxs'][0]:
            return
        for ex in self.data:
            ex['ctxs'].sort(key=lambda x: float(x['score']), reverse=True)

    def get_example(self, index):
        return self.data[index]


In [38]:
model_name = 't5-large'
text_maxlength = 200
answer_maxlength = 20
eval_data = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25_SELECTION/strict_positive_strict_damaging_remove_damage_irrelevant_relevant.json'

In [39]:
tokenizer = transformers.T5Tokenizer.from_pretrained(model_name)

In [40]:
n_context = 5

In [41]:
collator_function = Collator(text_maxlength, tokenizer, n_context)

In [42]:
eval_examples = load_data(
        eval_data, 
)

In [43]:
len(eval_examples)

2539

In [44]:
eval_dataset = Dataset(
    eval_examples, 
    n_context, 
)

In [45]:
# i = 0
# pprint(len(eval_dataset[i]['passages']))
# pprint(eval_dataset[i])

In [46]:
eval_sampler = SequentialSampler(eval_dataset) 

In [47]:
eval_dataloader = DataLoader(
    eval_dataset, 
    sampler=eval_sampler, 
    batch_size=2,
    collate_fn=collator_function
)

In [48]:

for a in eval_dataloader:
#     print(a)
    print('=================================')

p['input_ids'] type
torch.int64
--------
p['attention_mask'] type
torch.int64
--------
p['input_ids'] type
torch.int64
--------
repli_tensor type
torch.int64
p['attention_mask'] type
torch.int64
--------
repli_tensor type
torch.int64
--------
p['input_ids'] type
torch.int64
--------
repli_tensor type
torch.int64
p['attention_mask'] type
torch.int64
--------
repli_tensor type
torch.int64
--------
p['input_ids'] type
torch.int64
--------
repli_tensor type
torch.int64
p['attention_mask'] type
torch.int64
--------
repli_tensor type
torch.int64
--------
p['input_ids'] type
torch.int64
--------
p['attention_mask'] type
torch.int64
--------
p['input_ids'] type
torch.int64
--------
repli_tensor type
torch.int64
p['attention_mask'] type
torch.int64
--------
repli_tensor type
torch.int64
--------
p['input_ids'] type
torch.int64
--------
repli_tensor type
torch.int64
p['attention_mask'] type
torch.int64
--------
repli_tensor type
torch.int64
--------
p['input_ids'] type
torch.int64
--------
repli

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 5 but got size 8 for tensor number 1 in the list.

In [61]:
a = torch.tensor([1,2,3])

In [62]:
a.dtype

torch.int64

In [63]:
a.to(torch.int32)

tensor([1, 2, 3], dtype=torch.int32)

In [65]:
a.dtype

torch.int64

In [66]:
b = a.to(torch.int32)

In [67]:
passage_ids = torch.tensor(passage_ids).to(torch.int64)

tensor([1, 2, 3], dtype=torch.int32)

In [ ]:
text_passages = [['question: which president of the united states was a boy scout title: Boy '
  'Scouts of America context: Rowe, Steven Spielberg, Mayor Michael Bloomberg, '
  'Secretary of Defense Robert Gates, and President Gerald Ford are just a '
  'small sample of Eagle Scouts. Since February 2019, girls have been eligible '
  'to earn ranks and merit badges including Eagle Scout, following the same '
  'requirements as boys. Since the minimum time required time to earn Eagle '
  'Scout is about 18 months, a time extension has been allowed, ostensibly to '
  'ease the transition into the era of the first female Eagle Scout '
  'candidates. All scouts who were older than 16 but'],
 ['question: which river separates the bronx in new york city from manhattan '
  'island title: The Bronx context: The Bronx The Bronx is the northernmost of '
  'the five boroughs of New York City, in the U.S. state of New York. It is '
  'south of Westchester County; northeast and east of Manhattan, across the '
  'Harlem River; and north of Queens, across the East River. Since 1914, the '
  'borough has had the same boundaries as Bronx County, the third-most densely '
  'populated county in the United States. The Bronx has a land area of and a '
  'population of 1,471,160 in 2017. Of the five boroughs,',
  'question: which river separates the bronx in new york city from manhattan '
  'island title: New York City context: tidal strait—flows from Long Island '
  'Sound and separates the Bronx and Manhattan from Long Island. The Harlem '
  'River, another tidal strait between the East and Hudson Rivers, separates '
  'most of Manhattan from the Bronx. The Bronx River, which flows through the '
  'Bronx and Westchester County, is the only entirely fresh water river in the '
  "city. The city's land has been altered substantially by human intervention, "
  'with considerable land reclamation along the waterfronts since Dutch '
  'colonial times; reclamation is most prominent in Lower Manhattan, with '
  'developments such']]

In [ ]:
len(text_passages)

In [ ]:
max_length = 200
# t

In [ ]:
passage_ids, passage_masks = [], []

In [ ]:
t_passage = text_passages[1]

In [ ]:
t_passage

In [ ]:
p = tokenizer.batch_encode_plus(
            t_passage,
            max_length=max_length,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True
)

In [ ]:
p.keys()

In [ ]:
p['input_ids'].shape

In [ ]:
pprint(p['input_ids'])

In [ ]:
pprint(p['attention_mask'])

In [ ]:
# p_ = p['input_ids'][None]

In [ ]:
p['input_ids'].shape

In [ ]:
# replica = torch.zeros_like(p['input_ids'])

In [ ]:
n_contxt = 5

In [ ]:
cur_ctx = p['input_ids'].shape[0]

In [ ]:
cur_ctx

In [ ]:

repl_lst = []
while cur_ctx < n_context:
    cur_ctx += 1
    print(cur_ctx)
    input_size = torch.empty(1, p['input_ids'].shape[1])
    replica = torch.zeros_like(input_size)
    repl_lst.append(replica)

In [ ]:
input_size.shape

In [ ]:
p['input_ids'].shape

In [ ]:
replica.shape

In [ ]:
len(repl_lst)

In [ ]:
# repl_lst

In [ ]:
repli_tensor = torch.cat(repl_lst, dim=0)

In [ ]:
repli_tensor.shape

In [ ]:
merg_tensor = torch.cat([p['input_ids'], repli_tensor], dim = 0)

In [ ]:
merg_tensor.shape

In [ ]:
repli_tensor.shape

In [ ]:
p['input_ids'].shape

In [ ]:
merg_tensor.shape

In [ ]:
p['input_ids'] = merg_tensor

In [ ]:
p['input_ids'].shape

In [ ]:
def encode_passages_upto_n_context(batch_text_passages, tokenizer, max_length, n_context):
    passage_ids, passage_masks = [], []
    for k, text_passages in enumerate(batch_text_passages):
        p = tokenizer.batch_encode_plus(
            text_passages,
            max_length=max_length,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True
        )
        
        cur_ctx = p['input_ids'].shape[0]
        if cur_ctx <= n_context:
            repl_lst = []
            while cur_ctx < n_context:
                cur_ctx += 1
                replica = torch.zeros_like(p['input_ids'])
                repl_lst.append(replica)
            repli_tensor = torch.cat(repl_lst, dim=0)
            merg_tensor = torch.cat([p['input_ids'], repli_tensor], dim = 0)
            p['input_ids'] = merg_tensor
            
        cur_ctx = p['attention_mask'].shape[0]
        if cur_ctx < n_context:
            repl_lst = []
            while cur_ctx < n_context:
                cur_ctx += 1
                replica = torch.zeros_like(p['attention_mask'])
                repl_lst.append(replica)
            repli_tensor = torch.cat(repl_lst, dim=0)
            merg_tensor = torch.cat([p['attention_mask'], repli_tensor], dim = 0)
            p['attention_mask'] = merg_tensor
        
        passage_ids.append(p['input_ids'][None])
        passage_masks.append(p['attention_mask'][None])

    passage_ids = torch.cat(passage_ids, dim=0)
    passage_masks = torch.cat(passage_masks, dim=0)
    return passage_ids, passage_masks.bool()

In [ ]:
replica = torch.zeros_like(p['input_ids'])

In [ ]:
torch.stack((p['input_ids'], replica), dim = 0)

In [ ]:
p['input_ids']

In [ ]:
pad_

In [ ]:
torch.stack((p['input_ids'], pad_), dim = 0)

In [ ]:
p['input_ids'].shape

In [ ]:
pad_.shape

In [ ]:
p_.shape

In [ ]:
passage_masks = p['attention_mask'][None]

In [ ]:
def encode_passages(batch_text_passages, tokenizer, max_length):
    passage_ids, passage_masks = [], []
    for k, text_passages in enumerate(batch_text_passages):
        p = tokenizer.batch_encode_plus(
            text_passages,
            max_length=max_length,
            pad_to_max_length=True,
            return_tensors='pt',
            truncation=True
        )
        passage_ids.append(p['input_ids'][None])
        passage_masks.append(p['attention_mask'][None])

    passage_ids = torch.cat(passage_ids, dim=0)
    passage_masks = torch.cat(passage_masks, dim=0)
    return passage_ids, passage_masks.bool()

In [ ]:
passage_masks.bool()

In [ ]:
type(a)

In [ ]:
a[0]

In [ ]:
print(a[1].shape)
print(a[1].dim())

In [ ]:
y = torch.squeeze(a[1], 0)

In [ ]:
tokenizer.decode(y.numpy())

In [ ]:
import numpy as np
import pathlib
from pprint import pprint
import pandas as pd
from util import utils
import re
import os

## Get dataset for incremental Test
- README.md method 2,3
    - Method1 describes previous ranking system

## Definitions of Positive, Relevant, Damaging, Irrelevant  with respect to retriever (check out Meeting#3.ipynb)
    - Previous Definition of relevant passages are vague.
    - What is the definition of relevance?
        - Passages are retrived from the step?
    - Here we are going to dicuss the definition of each paradigm

### Previous passages are divded into two categories with respect to query
### Assumption 
    - higher the similarity score higher the the performance on downstream tasks
        - similarity score has positive correlation with positive passage
    - low similarity socre considered to be irrelevant degrades the output
    - Therefore, retrieval is a process of extracting positive passage from the relevant set
    - by shrinking the search space by top-k     
####  1. Positive passages 
        : passages that include correct answer
        : sometimes multiple passages are required
#### 2. Relevant passages (Relevant set : R)
        : passages have high possibilities in containing corret answer with repect to query/claim
        : usually measured by similariy score
#### 3. Irrelevant passages (Irrelevant set : I) 
        : passages have low probabilities in containing corret answer with repect to query/claim
        : usually measured by similariy score
    


### New Definitions are now divided into three categories with respect to query

### Assumption
    - Previous works focus on the re-ranking of retrieved passages, which based on the assumption that taking top-k passages with higher similarity scores will show higher performance in the downstream tasks.
    - ; however, we have been shown that among those retrieved passages with higher similarity scores actually downgrades the final output which could have been produce the correct answer without them.
    - So those passages are what we called damaging passges that have to be removed not just re-ranked by its score.

#### 1. Positive passages 
        : passages that produce correct inference on downstream task
        : sometimes multiple passages are required
        
#### 2. Relevant passages = R1
    : passages retrieved from query that does not present adversarial effect when previous inference is correct
    : passages with high possibility inlucde corret answer with repect to query/claim
    : usually measured by similariy score
    : Subset of of R
    
#### 3. Damaging passages = D1
    : passages retrieved from query that produce wrong output when preivous inference is correct
    : passages with high possibility inlucde corret answer with repect to query/claim
    : HOWEVER it degrades the inference output
    : usually measured by similariy score
    : Subset of of R
    
#### 4. Irrelevant passages
    : passages retrieved from query that does not change the answer when previous inference is not correct
    : when there is preivous inference result and current output is not correct, we call it irrelevant
    - Passages retrieved by retriever that have no impacts for inference
    - usually measured by similariy score

## Incremental Testing Setting

### 3 Method (Check out README.md)

### Method 2 (2nd method of in README.md)
    - The sooner, The better approach
      - Keep the Positive Context in order
    - Since we testing it, let's keep sample size of 5
    - When there is no Exact Mathcing during the incremental inference e.g.) em_pattern = '00000'
        - Keep the whole context so that those cases will have False on Exact Match values
        
    - Patterns: 
        - first 1 : positive
        - first 0 : irrelevant 
            - we know that first is the positive context that includes answer
            - with that perspective, it is relevant in theory
            - ; however, dataset is created via BLEU score on two different corpus.
            - If FiD does not correctly infer the output, 
            - context what we concieved of artificial positive context is actually irrelevant to the query.
            - Also when realistic scenario, we don't know whether the first context contains the answer
        - 01 pattern : Positive 
        - 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
        - 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
        - 10 pattern : Damaging
        
        - 11 pattern : relevant vs positive
            Strict Positive
                Under strict rule : consider it as relevant ?(or irrelevant?)
                                in terms of strictly limiting the number of positive passages
            Naive Positive
                Under naive rule : consider it as positive
                                in that this would increase the number of positive passages
                
        - A00 pattern : irrelevant vs damaging
            if '1' does not occured in A, currnet passage is irrelevant
            if '1' occurred in A, current passage is damaging either irrelevant 
                Strict Damaging
                    Under strict rule : consider it as irrelevnat 
                                    in terms of strictly limiting the number of damaging passage
                Naive Damaging
                    Under naive rule : consider it as damaging 
                                    in that this would increase the number of damaging passages
        
    - Options1 : Removes only damaging
    - Options2 : Removes damaging + irrelevant
    - Options3 : Removes damaging + relevant
    - Options4 : Removes damaging + irrelevant + relevnat
        

### Total Trials 16
    1st Trial - Options1(Remove only Damaging) + Strict Positive + Strict Damaging
    2nd Trial - Options1(Remove only Damaging) + Strict Positive + Naive Damaging
    3rd Trial - Options1(Remove only Damaging) + Naive Positive + Strict Damaging
    4th Trial - Options1(Remove only Damaging) + Naive Positive + Naive Damaging
    
    5th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Strict Damaging
    6th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Naive Damaging
    7th Trial - Options2(Remove damaging + irrelevant) + Naive Positive + Strict Damaging
    8th Trial - Options2(Remove damaging + irrelevant) + Naive Positive + Naive Damaging
    
    9th Trial - Options3(Removes damaging + relevant) + Strict Positive + Strict Damaging
    10th Trial - Options3(Removes damaging + relevant) + Strict Positive + Naive Damaging
    11th Trial - Options3(Removes damaging + relevant) + Naive Positive + Strict Damaging
    12th Trial - Options3(Removes damaging + relevant) + Naive Positive + Naive Damaging
    
    13th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Strict Damaging
    14th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Naive Damaging
    15th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Strict Damaging
    16th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Naive Damaging
        
        
    - Since Options4 removes  "damaging + irrelevant + relevant"
        - there will be no difference between Strict Damaging and Naive Damaging

    13, 14th Trial -> Options4(Removes damaging + irrelevant + relevant) + Strict Positive
    15, 16th Trial -> Options4(Removes damaging + irrelevant + relevant) + Naive Positive
    
    - Since Options2 removes  "damaging + irrelevant"
        - there will be no difference in terms of the input between Strict Damaging and Naive Damaging
    5, 6th Trial -> Options2(Remove damaging + irrelevant) + Strict Positive -> new 5th
    7, 8th Trial -> Options2(Remove damaging + irrelevant) + Naive Positive -> new 6th
    - Since Options2 include both "positive + relevant"
        - There will be no difference in terms of the input between Strict Positive and Naive Positive
        - I found out during checking
    new 5th, new 6th -> Options2(Remove damaging + irrelevant) 
    
    - Options4(Removes damaging + irrelevant + relevant) + Strict Positive
        - means one positive only thus same as "pos1_ctx1" -> nope there might be non-consecutives
        - e.g.) 01011 -> 2 strict positive cases
        - BUT "pos1_ctx1" = baseline could be our baselines
    Options4(Removes damaging + irrelevant + relevant) + Strict Positive -> equal to "pos1_ctx1" 
    

    
### Final Total Trials 11
    1st Trial - Options1(Remove only Damaging) + Strict Positive + Strict Damaging
    2nd Trial - Options1(Remove only Damaging) + Strict Positive + Naive Damaging
    3rd Trial - Options1(Remove only Damaging) + Naive Positive + Strict Damaging
    4th Trial - Options1(Remove only Damaging) + Naive Positive + Naive Damaging
    
    5th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Strict Damaging 
        - The one also represents other three cases
            - Options2(Remove damaging + irrelevant) + Strict Positive + Naive Damaging
            - Options2(Remove damaging + irrelevant) + Naive Positive + Strict Damaging
            - Options2(Remove damaging + irrelevant) + Naive Positive + Naive Damaging
    
    6th Trial - Options3(Removes damaging + relevant) + Strict Positive + Strict Damaging
    7th Trial - Options3(Removes damaging + relevant) + Strict Positive + Naive Damaging
    8th Trial - Options3(Removes damaging + relevant) + Naive Positive + Strict Damaging
    9th Trial - Options3(Removes damaging + relevant) + Naive Positive + Naive Damaging
    
    10th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive 
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Strict Damaging
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Naive Damaging
        - almost equal to "pos1_ctx1" e.g.) 01011 -> 2 strict positive cases
        - "pos1_ctx1" = baseline could be our baselines
        
    11th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive    
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Strict Damaging
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Naive Damaging
    
    
### Estimation 
    - Since retrieved passages have high similarity score, retrieved passages are higly likely to form damaging 
    passages with repect to query. 
    - Minimizing the risk of exposing query to damaging passages would be favorabe to FiD
    - The simplest way to reduce to the probability of such occurences is limiting the number of input passages

    - I think below strategy will hold the best result 
        1) Options2(Remove damaging + irrelevant)
            * As we saw from result from FiD result with Random Sampling, 
            * FiD is powerful distinguish the positive from irrelevants.
            * Even though we loose stronger boundary that might be helpful to inference, 
            * It will be (frivolous/trivial/not consequential)
        2) Strict Positive (relevant) 
            * Strict Positive puts a rigorous boundary for passages, meaning less positive passages
        3) Naive Damaging(damaging) 
            * Unlike Strict Positive, we can set up a lenient standard for damaging to lessen the size of input. 
            * Increasing Damaging passages by definition results in decreasing number of inputs

### Method2
    - Same approach but in reciprocal order
      - We know that FiD is order invariant but this is for checking 
      - don't need to test on whole trials

# Input
    - result from 5-1
    - /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json

# Trials
    - Need to check FiD input when there are less ctxs than n_context

In [ ]:
input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')

### format
```python
{
    'id' : str()
    'answers' : list()
    'ctxs' : list(dict)
    'questions' : str()
}
```

#### Test Code 

### Checking on Final Total Trials 12

    1st Trial - Options1(Remove only Damaging) + Strict Positive + Strict Damaging (checked)
    2nd Trial - Options1(Remove only Damaging) + Strict Positive + Naive Damaging (checked)
    3rd Trial - Options1(Remove only Damaging) + Naive Positive + Strict Damaging (checked)
    4th Trial - Options1(Remove only Damaging) + Naive Positive + Naive Damaging (checked)
    
    5th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Strict Damaging (checked)
        - The one also represents other three cases
            - Options2(Remove damaging + irrelevant) + Strict Positive + Naive Damaging 
            - Options2(Remove damaging + irrelevant) + Naive Positive + Strict Damaging (checked)
            - Options2(Remove damaging + irrelevant) + Naive Positive + Naive Damaging 
    
    6th Trial - Options3(Removes damaging + relevant) + Strict Positive + Strict Damaging (checked)
    7th Trial - Options3(Removes damaging + relevant) + Strict Positive + Naive Damaging (checked)
    8th Trial - Options3(Removes damaging + relevant) + Naive Positive + Strict Damaging (checked)
    9th Trial - Options3(Removes damaging + relevant) + Naive Positive + Naive Damaging (checked)
    
    10th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive (checked)
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Strict Damaging (checked)
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Naive Damaging
        - almost equal to "pos1_ctx1" e.g.) 01011 -> 2 strict positive cases
        - "pos1_ctx1" = baseline could be our baselines
        
    11th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive (checked)  
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Strict Damaging
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Naive Damaging (checked)

In [ ]:

# # cnt_conv = 0
# # empty_temp = {
# #     'psg_id' : None,
# #     'text' : None,
# #     'title' : None
# # }

# output_format = []

# # 'strict', 'naive'  
# # i = 0
# option_p = 'naive'
# option_d = 'naive'
# option = 'op4'

# for instance in input_file:
#     template_dict = {}
#     template_dict['id'] = instance['id']
#     template_dict['answers'] = instance['answers']
#     template_dict['question'] = instance['question']
#     template_dict['em_pattern'] = instance['em_pattern']
                                   
#     em_pattern = instance['em_pattern']
    
# #     i += 1
# #     if i == 3000:
# #         break
#     print(f'em_pattern : {em_pattern}')
#     # when there is at least one EM in the accumulated inference
#     if em_pattern != '00000':   
#         new_ctx = []
        
#         # relevant vs positive
#         positve_ctx_lst = []
#         relevant_ctx_lst = []
        
#         # irrelevant vs damaging
#         damaging_ctx_lst = []
#         irrelevant_ctx_lst = []
        
#         # print
#         # matchint each em to ctx
#         for ind_em, ctx in zip(em_pattern, instance['ctxs']):
#             print(f'{ind_em} : \t {ctx}')
        
#         for idx_, ctx in enumerate(instance['ctxs']):
            
#             # checking current em
#             cur_em = em_pattern[idx_]
#             pre_em_pattern = em_pattern[:idx_]
# #             print('-----------')
# #             print(f'em_pattern : {em_pattern}')
# #             print(f'idx_ : {idx_}')
# #             print(f'cur_em : {cur_em}')
# #             print(f'pre_em_pattern : {pre_em_pattern}')
# #             print(f'not pre_em_pattern : {not pre_em_pattern}')
# #             print('-----------')
            
#             # first 1 : positive
#             if not pre_em_pattern and cur_em == '1':
#                 positve_ctx_lst.append(ctx)
                
#                 # print
#                 print('-----------')
#                 print(f'em_pattern : {em_pattern}')
#                 print(f'idx_ : {idx_}')
#                 print(f'cur_em : {cur_em}')
#                 print(f'1 first positive ctx : ')
#                 pprint(ctx)
#                 print('-----------')
                
#             # first 0 : irrelevant
#             elif not pre_em_pattern and cur_em == '0':
#                 irrelevant_ctx_lst.append(ctx)
                
#                 # print
#                 print('-----------')
#                 print(f'em_pattern : {em_pattern}')
#                 print(f'idx_ : {idx_}')
#                 print(f'cur_em : {cur_em}')
#                 print(f'0 first irrelevant ctx : \t ')
#                 pprint(ctx)
#                 print('-----------')
                
#             # 01 pattern : positive 
#             elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '1':
#                 positve_ctx_lst.append(ctx)
                
#                 # print
#                 print('-----------')
#                 print(f'em_pattern : {em_pattern}')
#                 print(f'idx_ : {idx_}')
#                 print(f'cur_em : {cur_em}')
#                 print(f'01 pattern : positive  : \t ')
#                 pprint(ctx)
#                 print('-----------')
                
#             # 10 pattern : damaging
#             elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '0':
#                 damaging_ctx_lst.append(ctx)
                
#                 # print
#                 print('-----------')
#                 print(f'em_pattern : {em_pattern}')
#                 print(f'idx_ : {idx_}')
#                 print(f'cur_em : {cur_em}')
#                 print(f'10 pattern : damaging : \t ')
#                 pprint(ctx)
#                 print('-----------')
                
#             # 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
#             elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '1':
#                 if option_p == 'strict':
#                     relevant_ctx_lst.append(ctx)
                    
#                     # print
#                     print('-----------')
#                     print(f'em_pattern : {em_pattern}')
#                     print(f'idx_ : {idx_}')
#                     print(f'cur_em : {cur_em}')
#                     print(f'11 pattern : strict : relevant \t ')
#                     pprint(ctx)
#                     print('-----------')
                
#                 elif option_p == 'naive':
#                     positve_ctx_lst.append(ctx)
                    
#                     # print
#                     print('-----------')
#                     print(f'em_pattern : {em_pattern}')
#                     print(f'idx_ : {idx_}')
#                     print(f'cur_em : {cur_em}')
#                     print(f'11 pattern : naive : positive \t ')
#                     pprint(ctx)
#                     print('-----------')
                    
#                 else:
#                     print('option_p should be either \'strict\' or \'naive\'')
#                     break
                    
#             # 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
#             elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '0':
#                 # if '1' does not occured in A, currnet passage is irrelevant
#                 if not '1' in pre_em_pattern:
#                     irrelevant_ctx_lst.append(ctx)
                    
#                     # print
#                     print('-----------')
#                     print(f'em_pattern : {em_pattern}')
#                     print(f'idx_ : {idx_}')
#                     print(f'cur_em : {cur_em}')
#                     print(f'00 pattern : 1 does not exist : irrelevant_ctx_lst \t ')
#                     pprint(ctx)
#                     print('-----------')
#                 # if '1' occurred in A, 
#                 else:
#                     # strict : consider it as irrelevnat 
#                     if option_d == 'strict':
#                         irrelevant_ctx_lst.append(ctx)
#                         # print
#                         print('-----------')
#                         print(f'em_pattern : {em_pattern}')
#                         print(f'idx_ : {idx_}')
#                         print(f'cur_em : {cur_em}')
#                         print(f'00 pattern : 1 exists : strict :irrelevant_ctx_lst \t ')
#                         pprint(ctx)
#                         print('-----------')
#                     # naive : consider it as damaging 
#                     elif option_d == 'naive':
#                         damaging_ctx_lst.append(ctx)
#                         # print
#                         print('-----------')
#                         print(f'em_pattern : {em_pattern}')
#                         print(f'idx_ : {idx_}')
#                         print(f'cur_em : {cur_em}')
#                         print(f'00 pattern : 1 exists : naive :damaging_ctx_lst \t ')
#                         pprint(ctx)
#                         print('-----------')
#                     else:
#                         print('option_d should be either \'strict\' or \'naive\'')
#                         break
                    
#         # op1 removes damages only
#         if option == 'op1':
#             new_ctx.extend(positve_ctx_lst)
#             new_ctx.extend(relevant_ctx_lst)
#             new_ctx.extend(irrelevant_ctx_lst)
            
#             print(f'option : {option}')
#             print(f'removes damages only')
#             print(f'damages')
#             pprint(damaging_ctx_lst)
#             print(f'rest_ctx')
#             pprint(new_ctx)
            
#         # op2 removes damaging + irrelevant
#         elif option == 'op2':
#             new_ctx.extend(positve_ctx_lst)
#             new_ctx.extend(relevant_ctx_lst)
            
#             print(f'option : {option}')
#             print(f'removes removes damaging + irrelevant')
#             print(f'damages')
#             pprint(damaging_ctx_lst)
#             print(f'irrelevant')
#             pprint(irrelevant_ctx_lst)
#             print(f'rest_ctx')
#             pprint(new_ctx)
            
#         # op3 : Removes damaging + relevant
#         elif option == 'op3':
#             new_ctx.extend(positve_ctx_lst)
#             new_ctx.extend(irrelevant_ctx_lst)
            
#             print(f'option : {option}')
#             print(f'removes removes damaging + relevant')
#             print(f'damages')
#             pprint(damaging_ctx_lst)
#             print(f'relevant_ctx_lst')
#             pprint(relevant_ctx_lst)
#             print(f'rest_ctx')
#             pprint(new_ctx)
            
#         # op4 : Removes damaging + irrelevant + relevant
#         elif option == 'op4':
#             new_ctx.extend(positve_ctx_lst)
            
#             print(f'option : {option}')
#             print(f'removes removes damaging + irrelevant + relevant')
#             print(f'damages')
#             pprint(damaging_ctx_lst)
#             print(f'irrelevant')
#             pprint(irrelevant_ctx_lst)
#             print(f'relevant_ctx_lst')
#             pprint(relevant_ctx_lst)
#             print(f'rest_ctx')
#             pprint(new_ctx)
            
#         else:
#             print('option should be op1, op2, op3, op4')
#             break
        
#         template_dict['ctxs'] = new_ctx
#         output_format.append(template_dict)
        
#     # when there is no EM in the accumulated inference
#     else:
#         print(f'em_pattern == 00000')
#         template_dict['ctxs']= instance['ctxs']
#         output_format.append(template_dict)
        
    
#     print('==============instance finished======================')
# #         template_dict['ctxs']= new_ctx
# #         output_format.append(template_dict)
        
# #     else:
# #         template_dict['ctxs']= instance['ctxs']
# #         output_format.append(template_dict)


# Implementation

In [ ]:
def build_data(input_file, option, option_p, option_d):
    '''
    input_file : incremental inference result from FiD from KILT-5-1
        path : /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json
        
    output : FiD input json format
    
    option(required) : removing strategies
        op1 : removes damages only
        op2 : removes damaging + irrelevant
        op3 : removes damaging + relevant
        op4 : removes damaging + irrelevant + relevant
        
    option_p(required) : positive passage selection options
        strict : strict positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is relevant
        naive : naive positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is positive
                
    option_d(required) : damaging passage selection options
        strict : strict negative
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is irrelevant
        naive : naive damaging
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is damaging
    
    '''
    
    output_format = []

    # 'strict', 'naive'  
    # option_p = 'naive'
    # option_d = 'naive'
    # option = 'op4'

    for instance in input_file:
        template_dict = {}
        template_dict['id'] = instance['id']
        template_dict['answers'] = instance['answers']
        template_dict['question'] = instance['question']
        template_dict['em_pattern'] = instance['em_pattern']

        em_pattern = instance['em_pattern']

        # when there is at least one EM in the accumulated inference
        if em_pattern != '00000':   
            new_ctx = []

            # relevant vs positive
            positve_ctx_lst = []
            relevant_ctx_lst = []

            # irrelevant vs damaging
            damaging_ctx_lst = []
            irrelevant_ctx_lst = []


            for idx_, ctx in enumerate(instance['ctxs']):

                # checking current em
                cur_em = em_pattern[idx_]
                pre_em_pattern = em_pattern[:idx_]


                # first 1 : positive
                if not pre_em_pattern and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # first 0 : irrelevant
                elif not pre_em_pattern and cur_em == '0':
                    irrelevant_ctx_lst.append(ctx)
                    
                # 01 pattern : positive 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # 10 pattern : damaging
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '0':
                    damaging_ctx_lst.append(ctx)

                # 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '1':
                    if option_p == 'strict':
                        relevant_ctx_lst.append(ctx)

                    elif option_p == 'naive':
                        positve_ctx_lst.append(ctx)

                    else:
                        print('option_p should be either \'strict\' or \'naive\'')
                        return 

                # 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '0':
                    # if '1' does not occured in A, currnet passage is irrelevant
                    if not '1' in pre_em_pattern:
                        irrelevant_ctx_lst.append(ctx)

                    # if '1' occurred in A, 
                    else:
                        # strict : consider it as irrelevnat 
                        if option_d == 'strict':
                            irrelevant_ctx_lst.append(ctx)

                        # naive : consider it as damaging 
                        elif option_d == 'naive':
                            damaging_ctx_lst.append(ctx)

                        else:
                            print('option_p should be either \'strict\' or \'naive\'')
                            return 

            # op1 removes damages only
            if option == 'op1':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)


            # op2 removes damaging + irrelevant
            elif option == 'op2':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)

            # op3 : Removes damaging + relevant
            elif option == 'op3':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)

            # op4 : Removes damaging + irrelevant + relevant
            elif option == 'op4':
                new_ctx.extend(positve_ctx_lst)

            else:
                print('option should be op1, op2, op3, op4')
                return 

            template_dict['ctxs'] = new_ctx
            output_format.append(template_dict)

        # when there is no EM in the accumulated inference
        else:
            template_dict['ctxs']= instance['ctxs']
            output_format.append(template_dict)
    
    print('==============instance finished======================')
    return output_format
        


In [ ]:
option_dict = {
    'op1' : 'remove_damage',
    'op2' : 'remove_damage_irrelevant',
    'op3' : 'remove_damage_relevant',
    'op4' : 'remove_damage_irrelevant_relevant',
}
# option = 'op4'

In [ ]:
option_p_dict = {
    'strict' : 'strict_positive',
    'naive' : 'naive_positive',
}
# option_p = 'strict'

In [ ]:
option_d_dict = {
    'strict' : 'strict_damaging',
    'naive' : 'naive_damaging',
}
# option_d = 'strict'

In [ ]:
input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')
output_path = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25_SELECTION'

In [ ]:
for o_ in option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{option_dict[option]}.json'
            
            output_file = os.path.join(output_path, filename)
            output_format = build_data(input_file, option, option_p, option_d)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            